In [1]:
# NYPD Preparation
# 1 - remove columns not relevant to exercise
# 2 - Remove all data outside of range 2015-01-01 to 2019-12-31
# 3 - Remove rows with NaN, (null), UNKNOWN values
# 4 - map values like hour, severity score, offense super categories
# 5 - merge in features from Mental Health Services and Population Census using a Geospatial Queries

# Output: ../data/NYPD_Complaint_Data_Historic_Cleaned_Reduced.csv

In [2]:
# check for required file

import os
from pathlib import Path

FILE_PATH = '../data/NYPD_Complaint_Data_Historic.csv'

file_path = Path(FILE_PATH)

# Check if the file exists
if file_path.exists():
    print(f"File found: {file_path}")
else:
    print(f"File not found: {file_path}. Please ensure the file is downloaded correctly.")



File found: ../data/NYPD_Complaint_Data_Historic.csv


In [3]:
# Load the dataset
import pandas as pd
import numpy as np

initial_df = pd.read_csv(FILE_PATH)

initial_df.info()

/var/folders/7b/c18lkd_95v345lvvmts66_k40000gn/T/ipykernel_85226/1355347080.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_df = pd.read_csv(FILE_PATH)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914838 entries, 0 to 8914837
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         object 
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  int64  
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 25  SUSP_SEX          

In [4]:
# Columns to drop
columns_to_drop = ['LAW_CAT_CD', 'CMPLNT_NUM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'PD_CD', 'PD_DESC', 'CRM_ATPT_CPTD_CD', 
                   'JURIS_DESC', 'JURISDICTION_CODE', 'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'LOC_OF_OCCUR_DESC', 
                   'RPT_DT', 'X_COORD_CD', 'Y_COORD_CD', 'TRANSIT_DISTRICT', 'Lat_Lon', 'KY_CD', 'ADDR_PCT_CD', 'STATION_NAME',
                   'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'PATROL_BORO']

# Drop the columns
initial_df = initial_df.drop(columns=columns_to_drop)
initial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914838 entries, 0 to 8914837
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CMPLNT_FR_DT   object 
 1   CMPLNT_FR_TM   object 
 2   OFNS_DESC      object 
 3   BORO_NM        object 
 4   PREM_TYP_DESC  object 
 5   Latitude       float64
 6   Longitude      float64
 7   VIC_AGE_GROUP  object 
 8   VIC_RACE       object 
 9   VIC_SEX        object 
dtypes: float64(2), object(8)
memory usage: 680.1+ MB


In [5]:
initial_df.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,OFNS_DESC,BORO_NM,PREM_TYP_DESC,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,12/05/2006,10:38:00,MURDER & NON-NEGL. MANSLAUGHTER,QUEENS,(null),NaN,NaN,25-44,WHITE HISPANIC,M
1,08/25/1973,12:15:00,MURDER & NON-NEGL. MANSLAUGHTER,MANHATTAN,MULTI DWELL - APT BUILD,NaN,NaN,45-64,BLACK,F
2,04/22/2006,16:50:00,MURDER & NON-NEGL. MANSLAUGHTER,BROOKLYN,(null),NaN,NaN,45-64,BLACK,M
3,07/20/2006,22:37:00,MURDER & NON-NEGL. MANSLAUGHTER,BROOKLYN,(null),NaN,NaN,18-24,BLACK,M
4,04/14/2006,02:37:00,MURDER & NON-NEGL. MANSLAUGHTER,BROOKLYN,(null),NaN,NaN,45-64,BLACK,M


In [6]:
# Remove older data
# range 2015-01-01 to 2019-12-31 is non pandemic years
# Alignment with Other Datasets

initial_df['CMPLNT_FR_DT'] = pd.to_datetime(initial_df['CMPLNT_FR_DT'], errors='coerce')

# Filter out records outside the range 2015-01-01 to 2019-12-31
initial_df = initial_df[
    (initial_df['CMPLNT_FR_DT'] >= pd.Timestamp('2015-01-01')) &
    (initial_df['CMPLNT_FR_DT'] <= pd.Timestamp('2019-12-31'))
]

print(f"Filtered data size for records between 2015 and 2019: {initial_df.shape[0]}")



initial_df = initial_df.drop(columns=['CMPLNT_FR_DT'])

initial_df.head()

Filtered data size for records between 2015 and 2019: 2348043


,CMPLNT_FR_TM,OFNS_DESC,BORO_NM,PREM_TYP_DESC,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
95,17:50:00,ROBBERY,MANHATTAN,BANK,40.762602,-73.985875,(null),UNKNOWN,D
96,19:20:00,ROBBERY,BROOKLYN,STREET,40.701593,-73.948747,<18,WHITE,M
97,23:00:00,RAPE,MANHATTAN,STREET,40.737203,-73.983273,<18,BLACK,F
98,14:40:00,GAMBLING,BRONX,STREET,40.831107,-73.930410,(null),UNKNOWN,E
99,23:00:00,RAPE,BRONX,HOSPITAL,40.810352,-73.924942,<18,BLACK,F


In [7]:
# Replace '(null)' with NaN
initial_df.replace('(null)', np.nan, inplace=True)

# Replace 'UNKNOWN' with NaN
initial_df.replace('UNKNOWN', np.nan, inplace=True)

# Count the number of NaN values per column
nan_counts = initial_df.isna().sum()

# Print the NaN counts
print(nan_counts)

initial_df = initial_df.dropna()

# Count the number of NaN values per column
nan_counts = initial_df.isna().sum()

# Print the NaN counts
print(nan_counts)

CMPLNT_FR_TM          0
OFNS_DESC          1277
BORO_NM              27
PREM_TYP_DESC       861
Latitude              1
Longitude             1
VIC_AGE_GROUP    691753
VIC_RACE         735567
VIC_SEX               6
dtype: int64
CMPLNT_FR_TM     0
OFNS_DESC        0
BORO_NM          0
PREM_TYP_DESC    0
Latitude         0
Longitude        0
VIC_AGE_GROUP    0
VIC_RACE         0
VIC_SEX          0
dtype: int64


In [8]:
print(f"Cleaned data size: {initial_df.shape}")

Cleaned data size: (1588789, 9)


In [9]:
# Extract the hour from the time column using .loc to avoid the warning
initial_df.loc[:, 'Hour'] = initial_df['CMPLNT_FR_TM'].str.split(':').str[0].astype(float)
initial_df = initial_df.drop(columns=['CMPLNT_FR_TM'])

# Map Severity Score
# Map Crimes to a severity
# Read the CSV file into a DataFrame
severity_df = pd.read_csv('../mappings/offense_severity_scores.csv')

severity_dict = severity_df.set_index('Offense')['Severity_Score'].to_dict()
initial_df['OFNS_DESC_Severity_Score'] = initial_df['OFNS_DESC'].map(severity_dict)

# Print the updated DataFrame to verify
initial_df.head()

,OFNS_DESC,BORO_NM,PREM_TYP_DESC,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Hour,OFNS_DESC_Severity_Score
96,ROBBERY,BROOKLYN,STREET,40.701593,-73.948747,<18,WHITE,M,19.0,8
97,RAPE,MANHATTAN,STREET,40.737203,-73.983273,<18,BLACK,F,23.0,10
99,RAPE,BRONX,HOSPITAL,40.810352,-73.924942,<18,BLACK,F,23.0,10
100,FELONY ASSAULT,QUEENS,OTHER,40.593680,-73.790074,25-44,WHITE,F,12.0,9
101,ROBBERY,BRONX,STREET,40.859853,-73.894368,<18,BLACK HISPANIC,M,20.0,8


In [10]:
import pandas as pd

# Load the crime categories from the CSV
category_df = pd.read_csv('../mappings/Crime_Categories.csv')

# Convert the DataFrame into a dictionary where each crime is mapped to its category
crime_to_category = {crime: category for category, crimes in category_df.groupby('Category')['Crime'] for crime in crimes.values}

# Function to categorize each offense
def categorize_crime(offense):
    # Check if the offense exists in the dictionary and return the category
    return crime_to_category.get(offense, "Other")

# Apply the categorization function to the 'OFNS_DESC' column or your relevant column name
initial_df['Crime_Category'] = initial_df['OFNS_DESC'].apply(categorize_crime)

other_crimes_df = initial_df[initial_df['Crime_Category'] == 'Other']

# Optionally, you can check how many such rows there are
print(f"Number of 'Other' category crimes: {len(other_crimes_df)}")


print(other_crimes_df['OFNS_DESC'].unique())

# Check the first few rows to verify the new column
initial_df.head()

Number of 'Other' category crimes: 0
[]


,OFNS_DESC,BORO_NM,PREM_TYP_DESC,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Hour,OFNS_DESC_Severity_Score,Crime_Category
96,ROBBERY,BROOKLYN,STREET,40.701593,-73.948747,<18,WHITE,M,19.0,8,Violent Crimes
97,RAPE,MANHATTAN,STREET,40.737203,-73.983273,<18,BLACK,F,23.0,10,Violent Crimes
99,RAPE,BRONX,HOSPITAL,40.810352,-73.924942,<18,BLACK,F,23.0,10,Violent Crimes
100,FELONY ASSAULT,QUEENS,OTHER,40.593680,-73.790074,25-44,WHITE,F,12.0,9,Violent Crimes
101,ROBBERY,BRONX,STREET,40.859853,-73.894368,<18,BLACK HISPANIC,M,20.0,8,Violent Crimes


In [11]:
# Count the number of NaN values per column
nan_counts = initial_df.isna().sum()

# Print the NaN counts
print(nan_counts)

OFNS_DESC                   0
BORO_NM                     0
PREM_TYP_DESC               0
Latitude                    0
Longitude                   0
VIC_AGE_GROUP               0
VIC_RACE                    0
VIC_SEX                     0
Hour                        0
OFNS_DESC_Severity_Score    0
Crime_Category              0
dtype: int64


In [12]:
initial_df.shape

(1588789, 11)

In [13]:
# Import the MongoDBGeoHandler class
# Map in Population and Services Data
from mongo_handler import MongoDBGeoHandler

mental_health_handler = MongoDBGeoHandler(db_name='nyc_data', 
                                          collection_name='mental_health_services')
print(mental_health_handler.count_all_records())
# Query for the number of records and average score within a 5000-meter radius of a specific point
result = mental_health_handler.count_records_within_radius(40.730610, -73.935242, 5000)
print(result)
census_handler = census_handler = MongoDBGeoHandler(db_name='nyc_data', 
                                                    collection_name='census_data', 
                                                    score_column_name='TotalPop')
print(census_handler.count_all_records())
result = census_handler.count_records_within_radius(40.730610, -73.935242, 5000)
print(result)

490
{'count': 85, 'average_score': None}
18052
{'count': 1166, 'average_score': 3883.8902229845626}


In [14]:
from tqdm.notebook import tqdm
import pandas as pd

# Initialize tqdm for pandas integration
tqdm.pandas()

def fetch_data(row):
    lat, lon = row['Latitude'], row['Longitude']
    radius = 1000  # 1KM radius
    census_result = census_handler.count_records_within_radius(lat, lon, radius)
    mental_health_result = mental_health_handler.count_records_within_radius(lat, lon, radius)
    
    # Ensure defaults if None is returned
    census_avg_score = census_result.get('average_score', 0) or 0
    census_count = census_result.get('count', 0) or 0
    mh_count = mental_health_result.get('count', 0) or 0
    
    # Calculate and store results
    row['local_population_count_1km'] = census_avg_score * census_count
    row['mental_health_service_count_1km'] = mh_count
    
    return row

# Perform for sample of data for sanity check
sample_df = initial_df.head().progress_apply(fetch_data, axis=1)
sample_df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,OFNS_DESC,BORO_NM,PREM_TYP_DESC,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Hour,OFNS_DESC_Severity_Score,Crime_Category,local_population_count_1km,mental_health_service_count_1km
96,ROBBERY,BROOKLYN,STREET,40.701593,-73.948747,<18,WHITE,M,19.0,8,Violent Crimes,201434.0,4
97,RAPE,MANHATTAN,STREET,40.737203,-73.983273,<18,BLACK,F,23.0,10,Violent Crimes,310938.0,20
99,RAPE,BRONX,HOSPITAL,40.810352,-73.924942,<18,BLACK,F,23.0,10,Violent Crimes,224242.0,12
100,FELONY ASSAULT,QUEENS,OTHER,40.593680,-73.790074,25-44,WHITE,F,12.0,9,Violent Crimes,214372.0,0
101,ROBBERY,BRONX,STREET,40.859853,-73.894368,<18,BLACK HISPANIC,M,20.0,8,Violent Crimes,229936.0,8


In [ ]:
# Apply to all data
initial_df = initial_df.progress_apply(fetch_data, axis=1)

# Check the output
initial_df.head()

  0%|          | 0/1588789 [00:00<?, ?it/s]

In [ ]:
# Save the initial_df to a CSV file
initial_df.to_csv('../data/NYPD_Complaint_Data_Historic_Cleaned_Reduced_Merged.csv.csv', index=False)

print("DataFrame saved to NYPD_Complaint_Data_Historic_Cleaned_Reduced_Merged.csv.csv")

In [ ]:
initial_df.describe()